# Noisy circuits [[Link]](https://pennylane.ai/qml/demos/tutorial_noisy_circuits.html)


## Noisy operations
---

Noise can be separated into two categories.
* **Coherent noise** is described by unitary operations that maintain the purity of the output quantum state. <br>
A common source are systematic errors originating from imperfectly-calibrated devices that do not exactly apply the desired gates, e.g., applying a rotation by an angle $\phi+\epsilon$ instead of $\phi$.
* **Incoherent noise** is more problematic: it originates from a quantum computer becoming entangled with the environment, resulting in mixed states -- probability distributions over different pure states. <br>
Incoherent noise thus leads to outputs that are always random, regardless of what basis we measure in.

The purpose of PennyLane's `default.mixed` device is to provide native support for mixed states and for simulating noisy computations. 

Let's use `default.mixed` to simulate a simple circuit for preparing the Bell state $|\psi\rangle =\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$. <br>
We ask the QNode to return the expectation value of $Z_0\otimes Z_1$:

In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
dev = qml.device('default.mixed', wires=2)
print(f"Input state is  = \n{np.real(dev.state)}")

Input state is  = 
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [7]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


print(f"QNode output = {circuit():.4f}")

QNode output = 1.0000


The device stores the output state as a density matrix. In this case, the density matrix is equal to $|\psi\rangle\langle\psi|$, where $|\psi\rangle=\frac{1}{\sqrt{2}}|00\rangle+|11\rangle$.

In [8]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]


Incoherent noise is modelled by quantum channels.

Mathematically, a quantum channel is a linear, completely positve, and trace-preserving (CPTP) map. <br>
A convenient strategy for representing quantum channnels is to empoly Kraus operators $\{K_i\}$ satisfying the condition $\sum_iK_i^\dagger K_i=I$. <br>
For an initial state $\rho$, the output state after the action of a channel $\Phi$ is:
$$
\Phi(\rho) = \sum_i K_i\rho K_i^\dagger.
$$
Unitary transformations are special cases of quantum channels, like pure states are special cases of mixed states: $U\rho U^\dagger$.

More generally, the action of a quantum channel == applying the Kraus operator $K_i$ with some associated probability. <br>
More precisely, the channel applies the transformation $\frac{1}{p_i}K_i\rho K_i^\dagger$ with probability $p_i=\text{Tr}[K_i\rho K_i^\dagger]$. <br>
Quantum channels therefore represent a probability distribution over different possible transformations on a quantum state.

For example, consider the bit flip channel.<br>
It describes a transformation that flips the state of a qubit (X gate) with probability $p$ and leaves it unchanged with probability $1-p$. Its Kraus operators are
$$
K_0 = \sqrt{1-p}\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix},\\
K_1 = \sqrt{p} \begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}.
$$
This channel can be implemented in PennyLane using the `qml.BitFlip` operation.

Let us see what happens when this type of noise acts on both qubits in the circuit.!

In [9]:
@qml.qnode(dev)
def bitflip_circuit(p):
    # Making the density matrix of a Bell state
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    # Generating bit-flip noise on each qubit.
    qml.BitFlip(p, wires=0)
    qml.BitFlip(p, wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [10]:
ps = [0.001, 0.01, 0.1, 0.2]
for p in ps:
    print(f"QNode output for bit flip probability {p} is {bitflip_circuit(p):.4f}")

QNode output for bit flip probability 0.001 is 0.9960
QNode output for bit flip probability 0.01 is 0.9604
QNode output for bit flip probability 0.1 is 0.6400
QNode output for bit flip probability 0.2 is 0.3600


We can use PennyLane to look under the hood and see the output state of the circuit for the largest noise parameter.

In [11]:
print(f"Output state for bit flip probability {p} is \n{np.real(dev.state)}")

Output state for bit flip probability 0.2 is 
[[0.34 0.   0.   0.34]
 [0.   0.16 0.16 0.  ]
 [0.   0.16 0.16 0.  ]
 [0.34 0.   0.   0.34]]


---
#### Mathematical description of bit-flip channel on a Bell state $|\Psi^+\rangle$

$\rho=|\Psi^+\rangle\langle\Psi^+|$ with $|\Psi^+\rangle = \frac{1}{\sqrt{2}}|00\rangle+|11\rangle$.
$$
\Phi_1(\rho) = \sum_i (K_i\otimes I)\rho (K_i\otimes I)^\dagger
$$
$$
\Phi_2(\Phi_1(\rho)) = \sum_j\sum_i (I\otimes K_j)(K_i\otimes I)\rho (K_i\otimes I)^\dagger(I\otimes K_j)^\dagger
$$
$$
\Phi_{12}(\rho) = \sum_{i,j} (K_i\otimes K_j)\rho (K_i\otimes K_j)^\dagger \\
$$

In [16]:
p = 0.2

rho = np.array([[1/2,0,0,1/2],[0,0,0,0],[0,0,0,0],[1/2,0,0,1/2]])
K_0 = np.sqrt(1-p)*np.array([[1,0],[0,1]])
K_1 = np.sqrt(p)*np.array([[0,1],[1,0]])

$$
(K_0\otimes K_0)\rho (K_0\otimes K_0)^\dagger + (K_0\otimes K_1)\rho (K_0\otimes K_1)^\dagger + (K_1\otimes K_0)\rho (K_1\otimes K_0)^\dagger + (K_1\otimes K_1)\rho (K_1\otimes K_1)^\dagger
$$

In [18]:
np.dot(np.dot(np.kron(K_0,K_0), rho), np.kron(K_0,K_0).T) + np.dot(np.dot(np.kron(K_0,K_1), rho), np.kron(K_0,K_1).T) \
    + np.dot(np.dot(np.kron(K_1,K_0), rho), np.kron(K_1,K_0).T) + np.dot(np.dot(np.kron(K_1,K_1), rho), np.kron(K_1,K_1).T)

tensor([[0.34, 0.  , 0.  , 0.34],
        [0.  , 0.16, 0.16, 0.  ],
        [0.  , 0.16, 0.16, 0.  ],
        [0.34, 0.  , 0.  , 0.34]], requires_grad=True)

---

Besides the bit flip channel, PennyLane supports several other noisy channels: <br>
`PhaseFlip`, `AmplitudeDamping`, `GeneralizedAmplitudeDamping`, `PhaseDamping`, and the `DepolarizingChannel`.<br>
You can also build your own custom channel using the operation `QubitChannel` by specifying its Kraus operators, or even submit a pull request introducing a new channel.

Let us take a look at another example. <br>
The depolarizing channel is a generalization of the bit flip and phase flip channels, where each of the three possible Pauli errors can be applied to a single qubit.  <br>
Its Kraus operators are given by
$$
K_0 = \\
K_1 = \\
K_2 = \\
K_3 = \\
$$

## Channel gradients
---